In [51]:
import numpy as np
import sympy as sp

#### Constants (Peacemaker)

In [52]:
pi = 4.0 * np.arctan(1.0)
planck = 6.62606957e-34         # J s
avogadro = 6.0221413e23
kb = 1.3806488e-23              # J K^-1
speed_of_light = 299792458.0    # m s^-1
amu = 1.660538921e-27           # kg
gas_constant = avogadro*kb
hbar = planck/(2.0*pi)
global_eps = 1.0e-10

# Moment of inertia - process_coordinate_record : cluster.f90

The subroutine calculates the moments of inertia for a cluster.
For that, first the total mass of the cluster is calculated. 
Using the coordinates of the atoms each, the center of mass is determined and set as origin.
Then the inertia tensor is calculated.
This tensor is diagonalized to obtain the eigenvalues which correspond to the moments of inertia in x, y and z direction of the corresponding cluster.
The moments of inertia are saved, and the clusters are assigned as atom, linear or nonlinear

### Calculation of the total mass

In [53]:
# Provide masses for elements
def element_mass(element, mass):
    if element == 'H':
        mass = 1.008
    elif element == 'C':
        mass = 12.01
    elif element == 'O':
        mass = 16.00
    elif element == 'Cl':
        mass = 35.45
        
    return mass   

In [54]:
# Provided is a one array with masses of the atoms in the cluster
# and a second array with the coordinates of the atoms in the cluster
# The function calculates the center of mass of the cluster

def center_of_mass(nr_atoms, masses, coordinates):
    com = np.zeros(3)
    for i in range(nr_atoms):
        com += masses[i]*coordinates[i]
        
        
    com /= np.sum(masses)
    
    return com

In [55]:
# Set com as the origin of the coordinate system
def com_origin(nr_atoms, coordinates, com):
    for i in range(nr_atoms):
        coordinates[i] -= com
        
    return coordinates

#### Tests

Test - cluster (c1m1w3-7.xyz)</p>
20   $~~~~~$     $\textcolor{darkred}{\text{number of atoms}}$</br>
-39.771584979232   $~~~~~$   $\textcolor{darkred}{\text{energy (ignored)}}$ </br>
C    $~~~~~$   -1.16898944033827  $~~~$ -0.14805428493990  $~~~$  0.32745185542377        $~~~~~$   $\textcolor{darkred}{\text{atom type, coordinates}}$</br>
Cl   $~~~~~$   -2.34784356765353  $~~~$ -1.28928508067893  $~~~$  0.99930307590600 </br>
Cl   $~~~~~$   -0.65822727470503  $~~~$ -0.69129854923607  $~~~$ -1.27452373497366 </br>
H    $~~~~~$   -0.29281244953045  $~~~$ -0.09624032526792  $~~~$  0.98689811267938 </br>
Cl   $~~~~~$   -1.88998652011726  $~~~$  1.46443581796416  $~~~$  0.20551396958876 </br>
C    $~~~~~$    2.76478224676894  $~~~$ -2.07459581216681  $~~~$ -0.19885824933856 </br>
H    $~~~~~$    3.04017750865408  $~~~$ -2.64738864366591  $~~~$  0.69127857857294 </br>
O    $~~~~~$    3.01513896067189  $~~~$ -0.70150365130769  $~~~$ -0.02078584488278 </br>
H    $~~~~~$    1.71213083048118  $~~~$ -2.26384057006596  $~~~$ -0.43213510816834 </br>
H    $~~~~~$    3.37938957253550  $~~~$ -2.40571568513168  $~~~$ -1.03494010845601 </br>
H    $~~~~~$    2.47344440334455  $~~~$ -0.37364082813805  $~~~$  0.72736968185670 </br>
O    $~~~~~$    1.38749152659684  $~~~$  0.47508052337209  $~~~$  1.86216790536171 </br>
H    $~~~~~$    1.75428367438544  $~~~$  0.63637236685189  $~~~$  2.73283384502543 </br>
H    $~~~~~$    1.39549826682973  $~~~$  1.34971325441160  $~~~$  1.39534117990936 </br>
O    $~~~~~$    1.34486902387060  $~~~$  2.68669110647591  $~~~$  0.41214803245629 </br>
H    $~~~~~$    1.71731567397097  $~~~$  2.30907836219124  $~~~$ -0.41677149677762 </br>
H    $~~~~~$    0.47187877834489  $~~~$  3.01687474618809  $~~~$  0.18984991489869 </br>
O    $~~~~~$    2.14004917691883  $~~~$  1.26284234041995  $~~~$ -1.72480591317545 </br>
H    $~~~~~$    2.62134584522111  $~~~$  0.56485487037962  $~~~$ -1.23969806978062 </br>
H    $~~~~~$    1.40266719753673  $~~~$  0.82485404829678  $~~~$ -2.15680622354843 </br>

In [64]:
# Array containing the atom types
elements = ['C','Cl', 'Cl','H', 'Cl', 'C', 'H', 'O', 'H', 'H', 'H', 'O', 'H', 'H', 'O', 'H', 'H', 'O', 'H', 'H']
nr_atoms = len(elements)

# Array containing the masses of the atoms
masses = np.array([element_mass(element, 0.0) for element in elements])

# matrix containing the coordinates of the atoms
coordinates = np.array(
[[-1.16898944033827,   -0.14805428493990,    0.32745185542377],
[ -2.34784356765353,   -1.28928508067893,    0.99930307590600],
[ -0.65822727470503,   -0.69129854923607,   -1.27452373497366],
[ -0.29281244953045,   -0.09624032526792,    0.98689811267938],
[ -1.88998652011726,    1.46443581796416,    0.20551396958876],
[  2.76478224676894,   -2.07459581216681,   -0.19885824933856],
[  3.04017750865408,   -2.64738864366591,    0.69127857857294],
[  3.01513896067189,   -0.70150365130769,   -0.02078584488278],
[  1.71213083048118,   -2.26384057006596,   -0.43213510816834],
[  3.37938957253550,   -2.40571568513168,   -1.03494010845601],
[  2.47344440334455,   -0.37364082813805,    0.72736968185670],
[  1.38749152659684,    0.47508052337209,    1.86216790536171],
[  1.75428367438544,    0.63637236685189,    2.73283384502543],
[  1.39549826682973,    1.34971325441160,    1.39534117990936],
[  1.34486902387060,    2.68669110647591,    0.41214803245629],
[  1.71731567397097,    2.30907836219124,   -0.41677149677762],
[  0.47187877834489,    3.01687474618809,    0.18984991489869],
[  2.14004917691883,    1.26284234041995,   -1.72480591317545],
[  2.62134584522111,    0.56485487037962,   -1.23969806978062],
[  1.40266719753673,    0.82485404829678,   -2.15680622354843]])

# Calculate the center of mass
com = center_of_mass(nr_atoms, masses, coordinates)
print('Center of mass:', com)

# Set the center of mass as the origin of the coordinate system
coordinates = com_origin(nr_atoms, coordinates, com)
x = coordinates[:,0]
y = coordinates[:,1]
z = coordinates[:,2]
display('x-coordinates:', x, y, z)

Center of mass: [-0.04072005  0.07544383  0.04374451]


'x-coordinates:'

array([-1.12826939, -2.30712351, -0.61750722, -0.25209239, -1.84926647,
        2.8055023 ,  3.08089756,  3.05585902,  1.75285089,  3.42010963,
        2.51416446,  1.42821158,  1.79500373,  1.43621832,  1.38558908,
        1.75803573,  0.51259883,  2.18076923,  2.6620659 ,  1.44338725])

array([-0.22349812, -1.36472891, -0.76674238, -0.17168416,  1.38899199,
       -2.15003964, -2.72283247, -0.77694748, -2.3392844 , -2.48115952,
       -0.44908466,  0.39963669,  0.56092854,  1.27426942,  2.61124728,
        2.23363453,  2.94143092,  1.18739851,  0.48941104,  0.74941022])

array([ 0.28370734,  0.95555856, -1.31826825,  0.9431536 ,  0.16176946,
       -0.24260276,  0.64753407, -0.06453036, -0.47587962, -1.07868462,
        0.68362517,  1.81842339,  2.68908933,  1.35159667,  0.36840352,
       -0.46051601,  0.1461054 , -1.76855043, -1.28344258, -2.20055074])

### Diagonalization of symmetric 3x3 matrices (Direct calculation)

Starting with a symmetric 3x3 matrix.

$ \text{A} = \begin{pmatrix} a_{11} & a_{12} & a_{13} \\ a_{21} & a_{22} & a_{23} \\ a_{31} & a_{32} & a_{33} \end{pmatrix}$, </p>
$ \det(\text{A}) = a_{11}a_{22}a_{33} + 2a_{12}a_{23}a_{13} - a_{13}^2a_{22} - a_{23}^2a_{11} - a_{12}^2a_{33}$, </p>
$ \text{tr}(\text{A}^2) = a_{11}^2+a_{22}^2+a_{33}^2 + 2 (a_{12}^2+a_{23}^2+a_{13}^2)$, </p>
$ \text{tr}^2(\text{A}) = a_{11}^2+a_{22}^2+a_{33}^2 + 2 ( a_{11}a_{33}+ a_{11}a_{22}+ a_{22}a_{33})$ </p></br>

For the calculation of the eigenvalues, the characteristic polynomial is needed. </p>
$ \text{A} \nu = \alpha \nu $ </p>
$\small p_{\text{A}}(\alpha) = \det(\text{A} - \alpha I) = -\alpha^3 + \alpha^2 (a_{11}+a_{22}+a_{33}) + \alpha (a_{12}^2+a_{23}^2+a_{13}^2 - a_{11}a_{33}- a_{11}a_{22}- a_{22}a_{33}) + a_{11}a_{22}a_{33} + 2a_{12}a_{23}a_{13} - a_{13}^2a_{22} - a_{23}^2a_{11} - a_{12}^2a_{33}$ </p>
$ 0 = \alpha^3 - \text{tr}(\text{A}) \alpha^2 - \frac{1}{2} (\text{tr}(\text{A}^2)-\text{tr}^2(\text{A})) - \det(\text{A})$ </p> </br>
    
This is solved via the trigonometric solution. </p>
$ \text{B} \mu = \beta \mu $ </p>
$ \text{A} = p \text{B} + q \text{I}~~~~$ with I being the identity matrix.</p></br>
A and B have the same eigenvalues if: </p>
$ \alpha = p \beta + q$ </p></br>
$ q = \text{tr}(\text{A})/3~~~~$ and $~~~~p = \sqrt{\frac{\text{tr}(\text{A}-q\text{I})^2}{6}}$ </p> 
$ \det(\beta \text{I} - B) = \beta^3 -3\beta -\det(B)$ </p></br>

Substitution and simplification:</p>
$\beta = 2\cos(\theta)$ </p>
$cos(3\theta) = 4\cos^3(\theta) - 3\cos(\theta)$ </p>
Leads to: </p>
$ \cos(3\theta) = \det(B)/2 $ </p></br>

Thus the eigenvalues are: </p>
$ \beta = 2 \cos(\frac{1}{3} \arccos(\det(B)/2)+\frac{2k\pi}{3}) $, $~~~~k = 0,1,2$


In [57]:
# Calculates the eigenvalues of a 3x3 matrix
def diagonalization(a, eig):
   # a     : input     - 3x3 matrix
   # eig   : output    - array of the three eigenvalues
    
   p1 = a[0][1]**2 + a[0][2]**2 + a[1][2]**2
   if (p1 == 0):
      # The matrix is already in its diagonal form
      eig = np.diag(a)
   else:
      q = np.trace(a)/3
      p2 = (a[0][0]-q)**2 + (a[1][1]-q)**2 + (a[2][2]-q)**2 + 2*p1
      p = np.sqrt(p2/6)
      b = (a - q*np.identity(3))/p
      r = np.linalg.det(b)/2 # determinant
      print(r)

   # In exact arithmetic for a symmetric matrix  -1 <= r <= 1
   # but computation error can leave it slightly outside this range.
   if (r <= -1):
      phi = pi/3
   elif (r >= 1):
      phi = 0
   else:
      phi = np.arccos(r)/3

   # Calculation of the eigenvalues
      
   # the eigenvalues satisfy eig3 <= eig2 <= eig1
   eig[0] = q + 2 * p * np.cos(phi)
   eig[2] = q + 2 * p * np.cos(phi + (2*pi/3))
   eig[1] = 3 * q - eig[0] - eig[2]     # since trace(A) = eig1 + eig2 + eig3

   return(eig)


#### Check with an external python module

In [58]:
def diagonalization_sympy(a, eig):
    
   a = sp.Matrix(a)
   eig = a.eigenvals()
   
   # sort eigenvalues from biggest to smallest
   eig = sorted(eig, reverse=True)

   return(eig)

#### Tests

Test matrix </p>
$ \text{A} = \begin{pmatrix} 2 & 2 & -1 \\ 2 & 1 & 0 \\ -1 & 0 & 4 \end{pmatrix}~~~~~~~~~~~~~$ Eigenvalues: $2 - \sqrt{7},~~ 3,~~ 2 + \sqrt{7}$

In [59]:
# Matrix and initial eigenvalues
a = [[2,2,-1],[2,1,0],[-1,0,4]]
eig = np.zeros(3)

# Calculation with direct method
direct = diagonalization(a, eig)
print('Direct method : ',direct)

# Calculation with sympy
sympy = diagonalization_sympy(a, eig)
print('sympy         : ',sympy)
print()

# Check wether differences are smaller than 10^-10
diff = np.abs(direct - sympy)
for i in range(3):
    if (diff[i] < 1.0e-10):
        print('Eigenvalue ',i+1,' is the same')
    else:
        print('Eigenvalue ',i+1,' is different')

-0.6008383824567203
Direct method :  [ 4.64575131  3.         -0.64575131]
sympy         :  [2 + sqrt(7), 3, 2 - sqrt(7)]

Eigenvalue  1  is the same
Eigenvalue  2  is the same
Eigenvalue  3  is the same
